# Example of the model-in-service deployment pattern

In this example, we'll build a service that predicts a quality score for a red wine given the ID of the red wine. Specifically, when a user sends a product ID of a red wine, the service will use the ID to retrieve the chemical attributes from a database and call a model to predict the quality using the chemical attributes.

A recommended way to follow along is to read this documentation while working on the code in [app/main.py](./app/main.py).

Within this notebook, code snippets will be displayed with a description of what is going on. The same code can be found within the "app" directory.

## Prepare the dependencies
The red wine service needs to call a model. First, navigate to the MLflow service [http://mlflow-server.local](http://mlflow-server.local) and then to the "mlflow-minio-test" experiment. 

<img src="./images/mlflow-experiment.png" width=1000/>

Click the MLflow run (the "Start Time" column) that produced an ElasticNet model for wine quality prediction. Then copy the full S3 URI of the artifacts and assign the path to the "model_s3_uri" variable in the next code cell. 

<img src="./images/full-path.png" width=1000/>

Next, let's download the dependency requirements of the model.

In [ ]:
# Remember to change this to your own S3 URI
model_s3_uri = "s3://mlflow/12/9d75a172ed7543cd9619cb6ab5589258/artifacts/model"

In [ ]:
# Import packages

import mlflow
import os 

In [ ]:
# Configure MLflow
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://mlflow-minio.local"

# Configure the credentials needed for accessing the MinIO storage service
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"

# Download the requirements.txt file of the model and print the file's location
file_path = mlflow.pyfunc.get_model_dependencies(model_uri=model_s3_uri)
print(file_path)

The "requirements.txt" is saved under the "/tmp" directory (e.g., /tmp/tmpry8b66pc/requirements.txt). Let's move it to the "app" directory.

In [ ]:
!mv {file_path} ./app

We'll use [FastAPI](https://fastapi.tiangolo.com/) to implement the service and the service will need to load the model from the MLflow service, we also need to add the following dependencies to [requirements.txt](./app/requirements.txt). (boto3 is a package for managing an S3 storage service and uvicorn is a web server implementation for Python.)
```text
... (other model dependencies)
boto3~=1.28.85
fastapi~=0.103.2
uvicorn[standard]~=0.23.2
```

To avoid messing up our "mlops_eng" Python environment used for doing the assignments, let's create a new conda environment for running this example. Run the following commands in a new terminal session. (You don't need to change the Python environment of this notebook.)
```bash
# Create a new conda environment
conda create -n model_in_svc -yf python==3.10
conda activate model_in_svc

# Suppose you're in the same directory as this notebook (i.e., 1_1_model_in_svc) in the terminal session
pip install -r app/requirements.txt
```

## Coding the server

Now, let's jump to the service code. 

In [app/main.py](./app/main.py), we begin by creating an instance of the `FastAPI` class. This instance will handle the user requests for the service. 

As previously described, the service will receive production IDs from users, so we need to implement a function to retrieve the corresponding chemical attributes based on a production ID. For simplicity, we're not going to use a real database but just mock the behavior using hard-coded data. This function will return one data point if the ID is even, otherwise another data point. 
```python
def get_chemical_attributes(product_id: int) -> pd.DataFrame:
    if product_id % 2 == 0:
        data = {"fixed acidity": [7.7], 
                "volatile acidity": [0.56], 
                ...
            }
    else:
        data = {"fixed acidity": [6.7], 
                "volatile acidity": [0.46], 
                ...
            }
    return pd.DataFrame(data)
```
Next, we need to load the model from MLflow so it can be used for prediction. The `load_model` function loads a red wine model from the MLflow service using the S3 URI of the model and stores it in Python function format. The model's S3 URI is defined in [app/config.py](./app/config.py) and you need to replace it with your own model's S3 URI. The `model` variable is made global so other functions can access it. By decorating a function with the `@app.on_event("startup")` decorator we ensure that the function is run when the service is initialized. This way we make sure that the model is ready before the service receives the first request. 
```python
@app.on_event("startup")
def load_model():
    # mlflow configuration
    ...

    global model
    # load the model
    model = mlflow.pyfunc.load_model(model_uri=MODEL_S3_URI)
```

Now, it's time to create a function to handle user requests. With `@app.get("/predict/{product_id}")` decorator. the `predict` function will be run when a user visits the endpoint "/predict" and appends a wine's product ID as a path parameter. This function first calls the `get_chemical_attributes` to get the chemical attributes, then calls the model to make the prediction, and finally return a dictionary (which FastAPI will convert into JSON format) containing the prediction.
```python
@app.get("/predict/{product_id}")
def predict(product_id: int):
    chemical_attrs = get_chemical_attributes(product_id)
    pred = model.predict(chemical_attrs)
    print(pred)
    return {"predicted score": pred[0]}
```
Finally, we use uvicorn (a web server implementation for Python) to spin the service up. The service will run at http://localhost:3000. 
```python
config = uvicorn.Config("main:app", port=3000, log_level="info")
server = uvicorn.Server(config)
server.run()
```
You can start the service using the following command:
```bash
# In the terminal where the "model_in_svc" conda environment is activated
# Suppose you are in 1_1_model_in_svc
python app/main.py
```

Let's send some requests to the service.

In [ ]:
import requests
response = requests.get("http://localhost:3000/predict/1")
print(response.json())

In [ ]:
response = requests.get("http://localhost:3000/predict/2")
print(response.json())

The responses should have the following content: `{'predicted score': ...}`.

---
You've seen how to embed a model inside an application. You can now go back to [1_deployment_patterns.ipynb](../1_deployment_patterns.ipynb) and explore another model deployment pattern. 